In [1]:
from datetime import datetime

from pytket import Circuit

from qnexus.client import auth, circuits, projects, jobs


In [2]:
project_ref = projects.submit(
    name=f"demo_{str(datetime.now())}", 
    description="a demo project", 
    properties={}
)

In [3]:
projects.add_property(
    project_ref, 
    name="Parameter_1", 
    property_type="float", 
    description="Parameter for my iterative algorithm", 
    required=False
)

In [4]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

In [5]:
my_circuit_ref = circuits.submit(
    circuit=c, 
    project=project_ref,
    name="My circuit",
    description="A new circuit",
    properties={"Parameter_1": 0.1}
)

In [7]:
new_circuit = my_circuit_ref.get_circuit()

In [8]:
new_circuit.H(0)

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; H q[0]; ]

In [9]:
my_new_circuit_ref = circuits.submit(
    circuit=new_circuit, 
    project=project_ref,
    name="My circuit 2",
    description="A newer circuit",
    properties={"Parameter_1": 0.34}
)

In [13]:
# Examine the circuit
my_new_circuit_ref.get_circuit()

[H q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; H q[0]; ]

In [10]:
compile_job_ref = jobs.submit_compile_job(
    name=f"compile_job_{str(datetime.now())}",
    circuits=my_circuit_ref,
    optimisation_level=1,
    project=project_ref
)

In [11]:
job_matches = jobs.jobs(name="compile", project_id=project_ref.id)

In [13]:
job_matches.jobs[-1]

JobRef(id=UUID('1a2523af-edb0-4f42-b8b6-e57b7e0c3f3c'), annotations=Annotations(name='compile-job-infundibuliform-ruby-high-spirited-cokeri', description=None, properties={}), job_type=<JobType.Compile: 'COMPILE'>, last_status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, last_message='Job has been submitted to Nexus.', project=ProjectRef(id=UUID('9c0a3ad3-7e87-489a-9510-feb4b86e9a79'), annotations=Annotations(name='placeholder', description=None, properties={})))

In [8]:
jobs.wait_for(compile_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2023, 12, 15, 8, 43, 37, 343553, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2023, 12, 15, 8, 43, 37, 34307, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [14]:
compiled_circuit_refs = jobs.compilation_results(compile_job_ref)

compiled_circuit_refs[0].get_circuit()

[TK1(0.5, 0.5, 0.5) q[0]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [10]:
execute_ref =jobs.submit_execute_job(
    name="My Execute Job",
    circuits=compiled_circuit_refs,
    project=project_ref,
    n_shots=[10]
)

In [11]:
await jobs.listen_job_status(execute_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2023, 12, 15, 8, 44, 8, 433566, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2023, 12, 15, 8, 44, 3, 233644, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [12]:
my_results = [item.get_result() for item in jobs.execution_results(execute_ref)]

In [13]:
my_results[0].get_result()

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[192]
 [192]
 [192]
 ...
 [192]
 [192]
 [  0]],state=None,unitary=None,density_matrix=None)